<a href="https://colab.research.google.com/github/TucspDretcm/Perm2b-AA/blob/main/Permanente_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b><u>Algebra Abstracta</u></b></h1>
2022-1

Permanente 2b

<h2>Integrantes:</h2>

*   Gabriel Ivan Rodriguez Postigo
*   Jaime Mateo Gutiérrez Muñoz
*   Alexander Carpio Mamani

# Euclides

In [1]:
def Euclides(a, b):
  if b == 0:
    return a
  else:
    return Euclides(b, a%b)

def Ext_Euclides(a, b):
  if b == 0:
    return (a, 1, 0)
  else:
    d, x_, y_ = Ext_Euclides(b, a%b)
    x, y = y_, x_ - int(a/b)*y_
    return (d, x, y)

def Inversa(a, n):
  if Euclides(a, n) == 1:
    mgd, x, y = Ext_Euclides(a, n)
    return x % n
  else:
    print("No existe inversa")

# Miller-Rabin

In [168]:
import random

# Iterativo
def EXPMOD(a, x, n):
  c = a % n
  r = 1
  while (x > 0):
    if x % 2 != 0:
      r = (r * c) % n
    c = (c * c) % n
    x = int(x/2)
  return r

def ES_COMPUESTO(a, n, t, u):
  # x = a**u % n    # ERROR! : OVERFLOW con números de varias cifras.
  x = EXPMOD(a,u,n)
  if x == 1 or x == n-1:
    return False  # n es posiblemente primo
  for i in range(t):
    x = EXPMOD(x,2,n)
    if x == n-1:
      return False  # n es posiblemente primo
  return True    # n es un número compuesto

def gen_random_a(n, s):  # genera "s" numeros random unicos en el rango de 2 y n-1, si no se pueden generar "s" números entonces retorna lo que tenga.
  randoms = []
  maximo = min(s, n-3)
  while len(randoms) < maximo:
    num = random.randint(2, n-1)
    if num not in randoms:
      randoms.append(num)
  return randoms

def MILLER_RABIN(n, s):
  t = 0
  u = n - 1
  while u % 2 == 0:
    u = u//2
    t = t + 1
  #for a in gen_random_a(n, s):
  for _ in range(s):
    a = random.randint(2, n-1)
    if ES_COMPUESTO(a, n, t, u):
      return False
  return True

def RANDOMBITS(b):
  n = random.randint(0, 2**b - 1)
  m = 2**(b-1) + 1
  return n | m    # el operador "|" nos permite hacer una operación binaria de "n" y "m" en binario(bin(n) v bin(m)).

def RANDOMGEN_PRIMOS(b, s):
  n = RANDOMBITS(b)
  while MILLER_RABIN(n, s) == False:
    n = n + 2
  return n

def GEN_PRIMO_SIGUIENTE(n, s):
  n = n + 1 - (n % 2)
  while MILLER_RABIN(n, s) == False:
    n = n + 2
  return n

# Implementar RSA KEY GENERATOR.

In [388]:
class RSA:
  def __init__(self, k):

    n = 1
    while True:
      self.e, self.d, self.n = self.RSA_KEY_GENERATOR(k, 10**n)
      m = random.randint(2,self.n-1)
      c = self.Cifrado(m)
      if m == self.Descifrado(c):
        break
      else:
        n += 1

    print("RSA (s={:}, e={:}, d={:}, n={:}, phi={:})".format(10**n, self.e, self.d, self.n, self.phi))

  def RSA_KEY_GENERATOR(self, k, s):
    if k<8:    # generar primos con bits menores al dividirlos entre 2 no abria muchos, ocurria un loop, o un error.
      k = 8
    p = RANDOMGEN_PRIMOS(k//2, s)
    q = RANDOMGEN_PRIMOS(k//2, s)
    while p==q:
      q = RANDOMGEN_PRIMOS(k//2, s)

    n, self.phi = p*q, (p-1)*(q-1)

    e = random.randint(2,n-1)
    while Euclides(e, self.phi)!=1:
      e = random.randint(2, n-1)
      
    d = Inversa(e, self.phi)
    return e, d, n

  def Cifrado(self, m):
    return EXPMOD(m, self.e, self.n)

  def Descifrado(self, c):
    return EXPMOD(c, self.d, self.n)
  
  def get_edn(self):
    return self.e, self.d, self.n

rsa = RSA(8)

RSA (s=10, e=29, d=29, n=143, phi=120)


In [393]:
rsa = RSA(59)

mensaje = 13
print("\nMensaje original: ", mensaje)
c = rsa.Cifrado(mensaje)
print("Cifrado: ", c)
print("Descifrado: ", rsa.Descifrado(c))

e,d,n = rsa.get_edn()

print("\ne*d mod phi =", e*d % rsa.phi)    # e*d mod n = 1 mod n
print("phi | (e*d - 1) = ", (e*d-1) % rsa.phi)   # phi | (e*d - 1)
print("mgd(e, phi)={:}\t mgd(d,phi)={:}".format(Euclides(e, rsa.phi), Euclides(d, rsa.phi)))

RSA (s=1000, e=159591720678573569, d=22724742037357541, n=224810904119749237, phi=224810903164223916)

Mensaje original:  13
Cifrado:  218831990259710270
Descifrado:  13

e*d mod phi = 1
phi | (e*d - 1) =  0
mgd(e, phi)=1	 mgd(d,phi)=1


# Crear un sistema RSA-64 (de k = 64 bits), y mostrar los valores para e, d y n. Generar una tabla con tres columnas m, c = P(m) y m' = S(c) - en teoría se cumple m' = m. Para las filas usar 10 valores números aleatorios distintos para m.

In [394]:
print(" m\t\t\t|\t c= P(m)\t\t|\t m' = S(c)")
print("-"*85)

for _ in range(10):
  m = random.randint(2, n-1)
  c = rsa.Cifrado(m)
  semi_m = rsa.Descifrado(c)
  print(" {:}\t|\t {:}\t|\t {:}".format(m, c, semi_m))


 m			|	 c= P(m)		|	 m' = S(c)
-------------------------------------------------------------------------------------
 176766408831992707	|	 61408374729407702	|	 176766408831992707
 183597512350914172	|	 195889659501958673	|	 183597512350914172
 136068147220167639	|	 38581632321644873	|	 136068147220167639
 44545726954338757	|	 212320741186060674	|	 44545726954338757
 159526595081864709	|	 64698867816577206	|	 159526595081864709
 102238157482647417	|	 198809144261950100	|	 102238157482647417
 42566669424109485	|	 173690110227850276	|	 42566669424109485
 187235573050613406	|	 191513237696801722	|	 187235573050613406
 70922504592093835	|	 160177844572460541	|	 70922504592093835
 129506786751827790	|	 66997309505938865	|	 129506786751827790


<h2>EXTRA: Frases o palabras Cifradas y Descifradas.</h2>

In [418]:
def Cifrado(msg):
  cade = ""
  data = []
  for m in msg:
    me = rsa.Cifrado(ord(m))    # pasamos el valor en ascci de la palabra a el Cifrado RSA
    encrip = me % ord('A') + 65   # rango [65, 122] donde 65='A' y 122='z'
    cade += str("%c" % encrip)   # convertimos de decimal a word ascci
    data.append(me)
  return cade, data

def Descifrado(msg, data):
  cade = ""
  for i in range(len(msg)):
    med = rsa.Descifrado(data[i])
    cade += str("%c" % med)
  return cade

words = ["Hola", "Juego", "pichanga", "lemur", "DBP", "Algebra", "Hulk", "Mina", "Gaa", "UCSP"]
for word in words:
  word_2, data = Cifrado(word)
  word_3 = Descifrado(word_2, data)
  print("(Original={:}, Cifrado={:}, Descifrado={:})".format(word, word_2, word_3))

(Original=Hola, Cifrado=qnb~, Descifrado=Hola)
(Original=Juego, Cifrado=HFQCn, Descifrado=Juego)
(Original=pichanga, Cifrado=CMBU~YC~, Descifrado=pichanga)
(Original=lemur, Cifrado=bQ|FO, Descifrado=lemur)
(Original=DBP, Cifrado=NSe, Descifrado=DBP)
(Original=Algebra, Cifrado=LbCQbO~, Descifrado=Algebra)
(Original=Hulk, Cifrado=qFb|, Descifrado=Hulk)
(Original=Mina, Cifrado=QMY~, Descifrado=Mina)
(Original=Gaa, Cifrado=C~~, Descifrado=Gaa)
(Original=UCSP, Cifrado=NZTe, Descifrado=UCSP)
